In [ ]:
## Import Python libraries ## 

import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import xarray as xr


%matplotlib inline
%reload_ext xsimlab.ipython


import fastscape

from orographic_precipitation.fastscape_ext import precip_model
from fastscape.processes import Bedrock

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

In [ ]:
## Build xarray-simlab model ##
SouthernAndesLEM = precip_model
SouthernAndesLEM = precip_model.drop_processes(['init_topography'])
SouthernAndesLEM = SouthernAndesLEM.update_processes({'bedrock': Bedrock})

SouthernAndesLEM

In [ ]:
## Visualise xarray-simlab ##
SouthernAndesLEM.visualize(show_inputs='True')

In [ ]:
## Model dimensions/grid dimensions ##
nx = 100
ny = 100
nn = nx * ny

xl = 100.e3
yl = 100.e3

BoundaryCondition = ['looped', 'looped', 'fixed_value', 'fixed_value']

## Model time and time stepping ##
EndTime_1 = 5e6
EndTime_2 = 5e6
EndTime_3 = 8e6
TimeSteps_1 = 10001
TimeSteps_2 = 10001
TimeSteps_3 = 16001

# EndTime = 1e6
# TimeSteps = 1001
ModelTime_1 = np.linspace(0., EndTime_1, TimeSteps_1)
ModelTime_2 = np.linspace(0., EndTime_2, TimeSteps_2)
ModelTime_3 = np.linspace(0., EndTime_3, TimeSteps_3)

PlotStep = 100

## Tectonics ##
k_coef= 2.5e-5
area_exp = 0.4
slope_exp = 1.0
diffusion_diffusivity = 0.


## Orographic ##
lapse_rate = -5.4 #mean lapse rate of area not too far away 
lapse_rate_m = -6.5 #left default
ref_density = 7.4e-3  #left default
rainfall_frequency = 5 #could not find high resolution data for hourly rainfall in the area of patagonia i was interested in looking at so just and estimate
latitude = 46  #latitude around the area north in the Patagonian Andes
precip_base =  1e-4 #not sure if I should change this or not
wind_speed = 10 #base to start with 
wind_dir = 270  # estimated direction given weather info 
precip_min = 0.1 #chatgpt value so could be wrong
conv_time = 2000 #2000 thesis value
fall_time = 2000 #2000 thesis value
nm = 0.003  #0.03 #thesis value
hw = 5000  #


## Initial topography ##
WhiteNoiseLevel = 50 # in [m], amplitude of noise

## Seed for Reproducibility ##
RandomSeed = 1410

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define grid size
nx, ny = 100, 100  # Example values; adjust as needed
nn = nx * ny  # Total number of nodes

# Initialize an array to store elevations
InitialTopography = np.zeros(nn)

# Assign elevations based on node indices (using slicing and grid structure)
InitialTopography[:int(0.25 * nn)] = 400    # First 25% of nodes
InitialTopography[int(0.25 * nn):int(0.5 * nn)] = 3375  # Next 25% of nodes
InitialTopography[int(0.5 * nn):int(0.75 * nn)] = 2500  # Next 25% of nodes
InitialTopography[int(0.75 * nn):] = 1000  # Last 25% of nodes

# Adding random noise to the elevations
np.random.seed(42)  # For reproducibility
WhiteNoiseLevel = 50  # Standard deviation of noise
Noise = np.random.normal(0, WhiteNoiseLevel, nn)
InitialTopography += Noise  # Add noise to the elevations

# Reshape the 1D array to 2D with shape (ny, nx)
InitialTopography_2D = InitialTopography.reshape((ny, nx))

# Define first stage uplift rates for each region
UpliftRates = np.zeros(nn)  # Initialize uplift rates array

# Assign uplift rates to regions (using slicing or fractions of the total nodes)
UpliftRates[:int(0.25 * nn)] = 0.01    # First 25% of nodes
UpliftRates[int(0.25 * nn):int(0.5 * nn)] = 0.041  # Next 25% of nodes
UpliftRates[int(0.5 * nn):int(0.75 * nn)] = 0.035  # Next 25% of nodes
UpliftRates[int(0.75 * nn):] = 0.02  # Last 25% of nodes

# Adding random noise to the uplift rates for realism (optional)
np.random.seed(42)  # For reproducibility
UpliftNoise = np.random.normal(0, 0.005, nn)  # Small noise in uplift rates
UpliftRates += UpliftNoise  # Add noise to the uplift rates

# Reshape the 1D array to 2D with shape (ny, nx)
UpliftRates_2D = UpliftRates.reshape((ny, nx))

# Rotate both topography and uplift rates 90 degrees anti-clockwise
RotatedTopography_2D = np.rot90(InitialTopography_2D, k=1)
RotatedUpliftRates_2D = np.rot90(UpliftRates_2D, k=1)

In [ ]:
import os

# Directory to save Zarr files
output_dir = "simulation_outputs"
os.makedirs(output_dir, exist_ok=True)

# Define conv_time values to test
conv_time_values = [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]  # Conv_time in seconds

# Loop through conv_time values and run simulations
for conv_time in conv_time_values:
    print(f"Running simulation for conv_time = {conv_time} seconds")

    # Set up the model with the current conv_time (same as before)
    in_ds_winds = xs.create_setup(
        model=SouthernAndesLEM,
        clocks={
            'time': ModelTime_1,
            'output': ModelTime_1[::PlotStep]
        },
        master_clock='time',
        input_vars={
            'grid': {'shape': [nx, ny], 'length': [xl, yl]},
            'boundary': {'status': BoundaryCondition},
            'topography': {'elevation': RotatedTopography_2D},
            'bedrock': {'elevation': RotatedTopography_2D},
            'uplift': {'rate': RotatedUpliftRates_2D},
            'spl': {'k_coef': k_coef, 'area_exp': area_exp, 'slope_exp': slope_exp},
            'diffusion': {'diffusivity': diffusion_diffusivity},
            'orographic': {
                'lapse_rate': lapse_rate,
                'lapse_rate_m': lapse_rate_m,
                'ref_density': ref_density,
                'rainfall_frequency': rainfall_frequency,
                'latitude': latitude,
                'precip_base': precip_base,
                'wind_speed': wind_speed,  # Fixed wind speed
                'wind_dir': wind_dir,
                'precip_min': precip_min,
                'conv_time': conv_time,  # Change conv_time
                'fall_time': fall_time,
                'nm': nm,
                'hw': hw,
            },
        },
        output_vars={'topography__elevation': 'time', 'orographic__precip_rate': 'time'}
    )

    # Run the simulation
    with xs.monitoring.ProgressBar():
        out_ds_winds = in_ds_winds.xsimlab.run(model=SouthernAndesLEM)
    
    # Store simulation results in Zarr format
    zarr_filename = os.path.join(output_dir, f"simulation_conv_time_{conv_time}.zarr")
    out_ds_winds.to_zarr(zarr_filename, mode='w')
    print(f"Saved results for conv_time = {conv_time} seconds to {zarr_filename}")


In [ ]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Directory containing Zarr files
output_dir = "simulation_outputs"

# List all Zarr files and sort by conv_time values
zarr_files = [f for f in os.listdir(output_dir) if f.startswith("simulation_conv_time_") and f.endswith(".zarr")]
zarr_files.sort(key=lambda f: float(f.split("_")[-1][:-5]))  # Extract and sort conv_time values

# Define the conv_time values to plot
desired_conv_times = np.linspace(200, 2000, 10)  # Adjust range based on actual values

# Load datasets and attach conv_time as a coordinate
datasets = []
conv_times = []

for zarr_file in zarr_files:
    conv_time = float(zarr_file.split("_")[-1][:-5])  # Extract conv_time from filename
    ds = xr.open_zarr(os.path.join(output_dir, zarr_file), chunks={})
    
    # Add conv_time as a coordinate
    ds = ds.assign_coords(conv_time=conv_time)
    
    datasets.append(ds)
    conv_times.append(conv_time)

# Concatenate datasets along the new conv_time dimension
combined_ds = xr.concat(datasets, dim="conv_time")

# Ensure conv_time is a dimension and coordinate
combined_ds = combined_ds.sortby("conv_time")

# Print available values for debugging
print("Available conv_time values in dataset:", combined_ds.conv_time.values)

# Fix: Use nearest selection for valid conv_time values
valid_conv_times = []
for ct in desired_conv_times:
    try:
        elevation_data = combined_ds.topography__elevation.isel(time=-1).sel(conv_time=ct, method="nearest").mean(dim='x')
        valid_conv_times.append(ct)
    except KeyError:
        print(f"Skipping {ct}: No matching data found.")

# Adjust number of rows and columns based on valid conv_time values
n_valid = len(valid_conv_times)
ncols = 4  # Define number of columns
nrows = (n_valid // ncols) + (1 if n_valid % ncols != 0 else 0)  # Calculate rows needed

# Create subplots dynamically based on valid conv_time values
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 10))
axes = axes.flatten()

# Initialize variables to track global min and max for consistent y-axis scaling
global_min_elevation = np.inf
global_max_elevation = -np.inf

# First, loop to find the global min and max of the mean topography elevations
for ct in valid_conv_times:
    elevation_data = combined_ds.topography__elevation.isel(time=-1).sel(conv_time=ct, method="nearest").mean(dim='x')
    global_min_elevation = min(global_min_elevation, elevation_data.min())
    global_max_elevation = max(global_max_elevation, elevation_data.max())

# Plot only for valid conv_time values
for idx, ct in enumerate(valid_conv_times):
    ax = axes[idx]
    time_step = str(combined_ds.time[-1].values)  # Extract the last time step value
    title = f"Conv Time: {ct:.2f} s\nTime Step: {time_step}"
    
    # Get the mean elevation for this conv_time at the last time step
    elevation_data = combined_ds.topography__elevation.isel(time=-1).sel(conv_time=ct, method="nearest").mean(dim='x')
    
    # Plot the elevation data as a line plot
    ax.plot(elevation_data, label='Mean Topography Elevation', color='blue')
    
    ax.set_title(title)
    ax.set_ylabel("Mean Topography Elevation")
    ax.set_xlabel("Spatial Index (x)")
    ax.set_ylim(global_min_elevation, global_max_elevation)
    ax.grid(True)

# Hide unused axes
for ax in axes[len(valid_conv_times):]:
    ax.axis('off')

plt.tight_layout()
plt.show()

# Close all datasets
for ds in datasets:
    ds.close()


In [ ]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Directory containing Zarr files
output_dir = "simulation_outputs"

# List all Zarr files and sort by conv_time
zarr_files = [f for f in os.listdir(output_dir) if f.startswith("simulation_conv_time") and f.endswith(".zarr")]
zarr_files.sort(key=lambda f: int(f.split("_")[-1][:-5]))  # Extract conv_time from filename

# Define specific conv_time values to plot
desired_conv_times = [200., 400., 600., 800., 1000., 1200., 1400., 1600., 1800., 2000.]  # Adjust based on your simulations

# Load datasets and attach conv_time as a coordinate
datasets = []
conv_times = []
time_steps = []

for zarr_file in zarr_files:
    conv_time = int(zarr_file.split("_")[-1][:-5])  # Extract value
    if conv_time in desired_conv_times:  # Only include selected values
        ds = xr.open_zarr(os.path.join(output_dir, zarr_file), chunks={})
        
        # Add conv_time as a coordinate
        ds = ds.assign_coords(conv_time=conv_time)
        
        datasets.append(ds)
        conv_times.append(conv_time)  # Append only desired values
        
        # Extract the last time step
        time_steps.append(ds.time.values[-1])

# Concatenate datasets along the new conv_time dimension
combined_ds = xr.concat(datasets, dim="conv_time")
combined_ds = combined_ds.sortby("conv_time")  # Ensure correct order

# Calculate grid layout for subplots
num_plots = len(conv_times)
num_cols = (num_plots + 1) // 2  
num_rows = (num_plots + num_cols - 1) // num_cols  

# 📌 Plot: Elevation Maps for Selected Conv Times at Final Time Step
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(40, 10), constrained_layout=True)
axes = axes.flatten()

# Loop over datasets and create plots
for ax, ds, conv_time, time_step in zip(axes, datasets, conv_times, time_steps):
    ds.topography__elevation.isel(time=-1).plot(ax=ax, vmin=0, vmax=10000, cmap='viridis')
    ax.set_title(f"Conv Time = {conv_time:.0f} sec | Time Step = {time_step}", fontsize=12)
    ax.set_xlabel("x")
    ax.set_ylabel("y")

# Hide any unused subplots
for i in range(len(conv_times), len(axes)):
    fig.delaxes(axes[i])

plt.savefig("Elevation_ConvTime_Selected.png")
plt.show()

# Close all datasets
for ds in datasets:
    ds.close()


In [ ]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Directory containing Zarr files
output_dir = "simulation_outputs"

# List all Zarr files and sort by conv_time values
zarr_files = [f for f in os.listdir(output_dir) if f.startswith("simulation_conv_time") and f.endswith(".zarr")]
zarr_files.sort(key=lambda f: int(f.split("_")[-1][:-5]))  # Extract conv_time values

# Define specific conv_time values to plot
desired_conv_times = [200., 400., 600., 800., 1000., 1200., 1400., 1600., 1800., 2000.]    # Adjust based on your simulations

# Load datasets and attach conv_time as a coordinate
datasets = []
conv_times = []
time_steps = []

for zarr_file in zarr_files:
    conv_time = int(zarr_file.split("_")[-1][:-5])  # Extract value
    if conv_time in desired_conv_times:  # Only include selected values
        ds = xr.open_zarr(os.path.join(output_dir, zarr_file), chunks={})
        
        # Add conv_time as a coordinate
        ds = ds.assign_coords(conv_time=conv_time)
        
        datasets.append(ds)
        conv_times.append(conv_time)  # Append only desired values
        
        # Extract the last time step
        time_steps.append(ds.time.values[-1])

# Concatenate datasets along the new conv_time dimension
combined_ds = xr.concat(datasets, dim="conv_time")
combined_ds = combined_ds.sortby("conv_time")  # Ensure correct order

# Calculate grid layout for subplots
num_plots = len(conv_times)
num_cols = (num_plots + 1) // 2  
num_rows = (num_plots + num_cols - 1) // num_cols  

# 📌 Plot: Precipitation Maps for Selected Conv Time Values at Final Time Step
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(40, 10), constrained_layout=True)
axes = axes.flatten()

# Loop over datasets and create plots
for ax, ds, conv_time, time_step in zip(axes, datasets, conv_times, time_steps):
    # Plot precipitation at the final time step
    precip_data = ds.orographic__precip_rate.isel(time=-1)
    precip_data.plot(ax=ax, vmin=0, vmax=10, cmap='Blues')
    ax.set_title(f"Conv Time = {conv_time:.0f} sec | Time Step = {time_step}", fontsize=12)
    ax.set_xlabel("x")
    ax.set_ylabel("y")

# Hide any unused subplots
for i in range(len(conv_times), len(axes)):
    fig.delaxes(axes[i])

plt.savefig("Precipitation_ConvTime_Selected.png")
plt.show()

# Close all datasets
for ds in datasets:
    ds.close()
